In [1]:
from artifacts_ipv6_sra_scanning.config import *
import pickle
import os
import ipaddress
import glob
from tqdm.auto import tqdm
import country_converter as coco
from parallel_pandas import ParallelPandas
ParallelPandas.initialize(n_cpu=24, split_factor=1, disable_pr_bar=False)

2025-09-29 14:23:57.042 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
files = glob.glob(f'{INTERIM_DATA_DIR}/*.csv')
files = glob.glob(f'{INTERIM_DATA_DIR}/*.csv')[:-1]

In [3]:
files

['/home/service/artifacts-ipv6-sra-scanning/data/interim/router-ips.csv',
 '/home/service/artifacts-ipv6-sra-scanning/data/interim/hitlist-ips.csv',
 '/home/service/artifacts-ipv6-sra-scanning/data/interim/itdk-ips.csv',
 '/home/service/artifacts-ipv6-sra-scanning/data/interim/ripe-ips.csv']

In [4]:
info_columns = ['Geo','AS-Number','BGP-Prefix','Org']

In [5]:
dfs = [pl.scan_csv(file) for file in tqdm(files)]

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
def concat_frames(dfs, labels, column,columns):
    if len(dfs) != len(labels):
        raise ValueError("The number of DataFrames must match the number of labels.")

    enriched_dfs = [
        df.select(columns).unique().with_columns(pl.lit(label).alias(column))
        for df, label in tqdm(zip(dfs, labels))
    ]

    return pl.concat(enriched_dfs, how="vertical")

In [7]:
labels=['SRA','Hitlist','ITDK','RIPE-Atlas','IXP']
labels=['SRA','Hitlist','ITDK','RIPE-Atlas']

In [8]:
column = 'AS-Number'
label='ASN'

In [9]:
tmp = concat_frames(dfs,labels,'source',column).collect().to_pandas()

0it [00:00, ?it/s]

In [17]:
vh.plot_upsetplot(tmp,column,'source',labels,f'upsetplot_sra_{column}',
                   label,title = '',bbox=(0.9,1),pickle_file=f'{INTERIM_DATA_DIR}/upsetplot_sra_{column}.pkl',minsubsetsize=0, colors=["#375E97", "#FB6542", "#c1195c", "#37975e",'#cc9900'],
                fontsize=10)

  0%|          | 0/10 [00:00<?, ?it/s]

/home/service/artifacts-ipv6-sra-scanning/reports/figures/upsetplot_sra_AS-Number.png
